In [11]:
from bs4 import BeautifulSoup
import requests
import re
import json
import pandas as pd
EXPORT_FOLDER = "./data/"

In [2]:
# url = "https://www.millenaire3.com/ressources/2023/changer-les-comportements-revue-et-enseignements-des-actions-metropolitaines"
# url2 = "https://www.millenaire3.com/ressources/2023/agir-sur-les-modes-de-vie-une-nouvelle-grille-d-analyse"
# url3 = "https://www.millenaire3.com/dossiers/2022/resilience-quelles-articulations-entre-acteurs-publics-et-initiatives-du-territoire"
# url4 = "https://www.millenaire3.com/dossiers/2022/resilience-quelles-articulations-entre-acteurs-publics-et-initiatives-du-territoire/institut-transitions-notre-but-c-est-de-former-ceux-qui-peuvent-agir-des-aujourd-hui"
# url5 = "https://www.millenaire3.com/dossiers/2022/resilience-quelles-articulations-entre-acteurs-publics-et-initiatives-du-territoire/les-alchimistes-lyon-alimentation-gaspillage-compostage-aucun-de-ces-sujets-ne-laisse-indifferent"
# url6 = "https://www.millenaire3.com/ressources/2023/crise-ecologique-le-metabolisme-de-l-economie-au-caeur-des-enjeux"
# url7 = "https://www.millenaire3.com/dossiers/2015/la-soierie-lyonnaise-entre-tradition-et-innovation-archives/la-soie-a-lyon-une-initiative-du-pouvoir-royal"
# urls = [url, url2, url3, url4, url5, url6, url7]

with open("./M3urls2.txt", "r") as f:
        urls = f.read().split("\n")

In [3]:
def get_metadata(soup, type, url, mult_items, errors):

    mult_items[url] = {"auteurs" : [], "tags" : [], "liens" : []}

    #TAGS
    tags_liste = soup.find_all("ul", {"class": "list-inline keywords"})
    tags = []
    for tag in tags_liste:
        a = tag.find_all("a")
        for tag in a:
            if re.search("tag", tag["href"]) != None:
                tags += tag.contents
    if len(tags) > 0:
        mult_items[url]["tags"] = tags
    else:
        mult_items[url]["tags"] = "NA"

    #AUTHORS
    if type == "Interview":
        liste = soup.find_all("ul", {"class": "list-inline keywords"})
        authors = []
        for elem in liste:
            a = elem.find_all("a")
            for author in a:
                if re.search("auteurs", author["href"]) != None:
                    authors.append(author.contents[0].strip())
    else:
        auth = soup.find_all("span", {"class" : "author-name"})
        authors = []
        for author in auth:
            author = repr(author.text).replace("\\xa0", " ").replace("\\t", "").replace("\\n", "").replace("'", "")
            if author not in authors:
                authors.append(author)
    if len(authors) > 0:
        mult_items[url]["auteurs"] = authors
    else:
        mult_items[url]["auteurs"] = "NA"
        
    #Liens
    page_links = soup.find_all("h3", {"class" : "h3 uppercase"})
    links = []
    for link in page_links:
        links.append("https://www.millenaire3.com"+link.a["href"])
    if len(links) > 0:
        mult_items[url]["liens"] = links
    else:
        mult_items[url]["liens"] = "NA"
    
    return(mult_items, errors)

In [4]:
def get_article_info(soup, type, url, container, errors):

    #RESUME
    if type == "Dossier":
        resume = soup.find("div", {"class" : "eztext-field"})
    elif type == "Interview":
        resume = soup.find("div", {"class" : "ezrichtext-field"})
    else: 
        resume = soup.find("div", {"class" : "eztext-field"})
    if len(resume) > 0:
        container["RESUME"].append(resume.text)
    else:
        errors["RESUME"].append(url)
        container["RESUME"].append("NA")

    #ARTICLE
    if type != "Interview":
        try:
            article = soup.find("div",  {"class" : "ezrichtext-field"}).text
        except:
            article = ""
    else:
        lines = soup.find_all("div", {"class" : "interview-item"})
        if len(lines) == 0:
            art = soup.find_all("div",  {"class" : "ezrichtext-field"})
            for info in art:
                if info != resume.text:
                    article = info.text
        else:
            article = ""
            for i, line in enumerate(lines):
                article += line.find("p", {"class" : "over-under-line"}).text + " "
                temp = line.find_all("div", {"class" : "ezrichtext-field"})
                for info in temp:
                    article += info.find("p").text
    if len(article) > 0:
        container["ARTICLE"].append(article.replace("\n", " "))
    else:
        errors["ARTICLE"].append(url)
        container["ARTICLE"].append("NA")

    #TITRE
    titre = soup.find("h1", {"class": "uppercase h1-special"}).contents[0]
    titre = titre.replace("\n", "").replace("\t", "").strip()
    if len(titre) > 0:
        container["TITRE"].append(titre)
    else:
        errors["TITRE"].append(url)
        container["TITRE"].append("NA")
        
    #ID - in db
    try:
        ID = soup.find("article", {"class" : "sidebar-block-classic sidebar-block-download"})
        ID = ID.a["href"].split("/")
        ID = ID[len(ID)-1]
        container["IDinDB"].append(ID)
    except:
        container["IDinDB"].append("NA")

    #DATE
    date = soup.find("span", {"class" : "publication_date"}).text
    if len(date) > 0:
        container["DATE"].append(date)
    else:
        errors["DATE"].append(url)
        container["DATE"].append("NA")
    
    #URL
    container["URL"].append(url)
    return(container, errors)

In [5]:
def scrap_page(url, container, mult_items, errors, ONLY_MULT = False):
    
    html = requests.get(url)
    if(html.status_code) != 200:
        print(html.status_code)
        errors["deadlink"].append(url)
        return(1)
    soup = BeautifulSoup(html.text, "html.parser")

    #TYPE
    type_liste = soup.find_all("p", {"class" : re.compile("^over-under")})
    if len(type_liste) > 0:
        type = ""
        for typ in type_liste:
            if re.search("Étude|Dossier|Article|Texte|Interview", typ.text) == None:
                continue
            else:
                if re.search("Interview|Texte", typ.text) == None:
                    type = typ.text.replace("\n", "").replace("\t", "").replace(" ", "")
                elif re.search("Texte", typ.text) != None:
                    type = "Texte"
                    txt_de = typ.text.replace("\n", "").replace("\t", "")
                elif re.search("Interview", typ.text) != None:
                    type = "Interview"
                    itw_de = typ.text.replace("\n", "").replace("\t", "")
    else:
        return(1)
    if len(type) > 0:
        container["TYPE"].append(type)
    else:
        errors["TYPE"].append(url)
        return(1)
    #END TYPE

    if ONLY_MULT == True:
        get_metadata(soup, type, url, mult_items, errors)
    else:
        get_article_info(soup, type, url, container, errors)
        get_metadata(soup, type, url, mult_items, errors)

    return(container, mult_items, errors)


In [7]:
ONLY_MULT = False

# container = {"URL" : [], 
#                 "TITRE" : [], 
#                 "IDinDB" : [], 
#                 "TYPE" : [], 
#                 "RESUME" : [], 
#                 "ARTICLE" : [],
#                 "DATE" : []}

# mult_items = {}

# errors = {
#         "TITRE" : [],
#         "TYPE" : [],
#         "RESUME" : [],
#         "ARTICLE" : [],
#         "DATE" : [],
#         "deadlink" : []
#     }

for i, url in enumerate(urls[7888:]):
    print(f"\r{i}/{len(urls)} : {url}", end = "")
    if re.search("/auteurs/|/tag/|partage_email|/generated_pdf|m3_search", url) == None and len(url) > 0:
        scrap_page(url, container, mult_items, errors, ONLY_MULT)
    else:
        pass
        #print("not an article")


for keys in container:
    print(keys, ":", len(container[keys]))


0/12367 : https://www.millenaire3.com/dossiers/Les-Cahiers-Millenaire-3-archive/Cahier-Millenaire-3-n-21-Une-agglomeration-competitive-et-rassemblee404
1/12367 : https://www.millenaire3.com/dossiers/Les-Cahiers-Millenaire-3-archive/Supplement-Cahier-Millenaire-3-n-21-Une-agglomeration-competitive-et-rassemblee404
2/12367 : https://www.millenaire3.com/dossiers/Les-Cahiers-Millenaire-3-archive/Cahier-Millenaire-3-n-22-L-ere-de-l-information-enjeux-economiques-sociaux-et-democratiques404
3/12367 : https://www.millenaire3.com/dossiers/Les-Cahiers-Millenaire-3-archive/Supplement-Cahier-Millenaire-3-n-22-Societe-de-l-information-l-agglomeration-lyonnaise-en-projets404
4/12367 : https://www.millenaire3.com/dossiers/Les-Cahiers-Millenaire-3-archive/Cahier-Millenaire-3-n-23-Cultes-cultures-et-laicite-sur-l-espace-commun404
5/12367 : https://www.millenaire3.com/dossiers/Les-Cahiers-Millenaire-3-archive/Supplement-Cahier-Millenaire-3-n-23-Paroles-et-panorama-des-religions-dans-l-agglomeration-lyo

In [9]:
for keys in container:
    print(len(container[keys]))

3971
3971
3971
3971
3971
3971
3971


In [12]:
# container["URL"].append("NA")
# container["TITRE"].append("NA")
# container["IDinDB"].append("NA")
# container["ARTICLE"].append("NA")
# container["DATE"].append("NA")
with open(EXPORT_FOLDER+"data_items.json", "w", encoding="utf8") as f:
    f.write(json.dumps(mult_items, indent=4))
tab = pd.DataFrame(container)
tab.to_csv(EXPORT_FOLDER+"data_article.csv")
tab

,URL,TITRE,IDinDB,TYPE,RESUME,ARTICLE,DATE
0,https://www.millenaire3.com/Interview/2023/j.-...,"J.-F. Spindler, chimiste : « Il y a de nouvell...",NA,Interview,Jean-Francis Spindler est directeur du Centre ...,"En tant que chimiste, quel regard portez-vous ...",15/02/2023
1,https://www.millenaire3.com/ressources/2023/le...,Le rapport à la propreté urbaine,539127,Étude,La propreté est une composante essentielle du ...,SommaireIntroduction1. La propreté : un concep...,16/05/2023
2,https://www.millenaire3.com/Interview/2023/jul...,"Julien Baltazar, doctorant en ingénierie : « L...",NA,Interview,Julien Baltazar réalise une thèse sur l’aide à...,La voiture électrique est aujourd’hui présenté...,01/02/2023
3,https://www.millenaire3.com/Interview/2023/dan...,Daniel Mugnier : « L’enjeu pour la technologie...,NA,Interview,Daniel Mugnier a travaillé avec passion dans l...,Comment a évolué la filière photovoltaïque au ...,27/03/2023
4,https://www.millenaire3.com/Interview/2023/ale...,"Alexandre Peutin, Matériauthèque de la Cité du...",NA,Interview,La matériauthèque de la Cité du Design de Sain...,Comment les matériaux présents dans la matéria...,23/02/2023
...,...,...,...,...,...,...,...
3966,https://www.millenaire3.com/ressources/La-pros...,"La prospective : une idée, plusieurs pratiques",51936,Texte,La prospective est née dans la pensée d’un hom...,"Gaston Berger, philosophe en action, est direc...",01/10/2012
3967,https://www.millenaire3.com/ressources/catholi...,"Catholicisme, et présence dans l’espace public",15261,Texte,L’Eglise catholique en France a perdu sa posit...,,01/01/2001
3968,https://www.millenaire3.com/Interview/2009/les...,Les dynamiques associatives et leur relation a...,NA,Interview,"Rencontre avec Jean-Pierre Aldeguer, directeur...","Vous avez maintenu, à travers des positions di...",22/07/2009
3969,https://www.millenaire3.com/ressources/les-usa...,Les usages administratifs de la forme associat...,35897,Texte,Jean-Pascal Bonhotal s’étonne que le phénomène...,Sommaire Les associations et l’administration ...,01/01/2002


In [39]:
i = 0
for URL in urls:
    if URL not in container["URL"] and URL not in errors["deadlink"] and "partage_email" not in URL and "/auteurs/" not in URL and "/tag/" not in URL:
        i += 1
        print(i, URL)

1 https://www.millenaire3.com
2 https://www.millenaire3.com/
3 https://www.millenaire3.com#content
4 https://www.millenaire3.com#main-menu
5 https://www.millenaire3.com#block-search
6 https://www.millenaire3.com/nos-flux-rss
7 https://www.millenaire3.com/societe
8 https://www.millenaire3.com/modes-d-action
9 https://www.millenaire3.com/transformation-urbaine
10 https://www.millenaire3.com/politiques-publiques
11 https://www.millenaire3.com/territoire
12 https://www.millenaire3.com/soutenabilite
13 https://www.millenaire3.com/actualites-a-la-une/le-rapport-a-la-proprete-urbaine
14 https://www.millenaire3.com/actualites-a-la-une/veille-m3-famille-du-concept-de-fraternite
15 https://www.millenaire3.com/actualites-a-la-une/le-spectacle-vivant-au-defi-de-la-jeunesse-et-du-numerique
16 https://www.millenaire3.com/actualites-a-la-une/crise-ecologique-le-metabolisme-de-l-economie-au-caeur-des-enjeux
17 https://www.millenaire3.com#carousel-example-generic
18 https://www.millenaire3.com#onglet_1

In [40]:
urls[7888]

'https://www.millenaire3.com/dossiers/Les-Cahiers-Millenaire-3-archive/Cahier-Millenaire-3-n-21-Une-agglomeration-competitive-et-rassemblee'

In [8]:
errors

{'TITRE': [],
 'TYPE': ['https://www.millenaire3.com',
  'https://www.millenaire3.com',
  'https://www.millenaire3.com',
  'https://www.millenaire3.com',
  'https://www.millenaire3.com',
  'https://www.millenaire3.com',
  'https://www.millenaire3.com',
  'https://www.millenaire3.com',
  'https://www.millenaire3.com',
  'https://www.millenaire3.com',
  'https://www.millenaire3.com',
  'https://www.millenaire3.com',
  'https://www.millenaire3.com',
  'https://www.millenaire3.com/',
  'https://www.millenaire3.com/',
  'https://www.millenaire3.com/',
  'https://www.millenaire3.com/',
  'https://www.millenaire3.com/',
  'https://www.millenaire3.com/',
  'https://www.millenaire3.com/',
  'https://www.millenaire3.com/',
  'https://www.millenaire3.com/',
  'https://www.millenaire3.com/',
  'https://www.millenaire3.com/',
  'https://www.millenaire3.com/',
  'https://www.millenaire3.com/',
  'https://www.millenaire3.com#content',
  'https://www.millenaire3.com#content',
  'https://www.millenaire